<a href="https://colab.research.google.com/github/jeanlucjackson/w266_final_project/blob/main/code/sandboxes/BB/table_metrics_by_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table for Metrics by Dataset

### Imports/Setup

In [ ]:
from os import listdir
from os.path import isfile, join

import csv
import json
import pprint

import pandas as pd
from tqdm import tqdm

In [ ]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
evaluation_root = "/content/drive/MyDrive/w266 NLP Final Project/Evaluation/"
filename = join(evaluation_root, "evaluation_database.json")

# Read JSON into dataframe
evaluation_df = pd.read_json(filename)

Set up ability to remove models with inference hyperparameters

In [ ]:
# Apply this mask to remove all samples from non default inference sets
default_hyperparameter_mask = [sample['defaults'] for sample in evaluation_df.hyperparameter]

### Cleanup dataframe

In [ ]:
# Clean up evaluation db df to only include columns needed for text analysis
clean_eval_df = evaluation_df[["nickname", "base_model", "trained_on", "tested_on", "hyperparameter", "target", "prediction", "bleu", "rougeL", "meteor", "bertscore-f1", "use"]]

clean_eval_df

,nickname,base_model,trained_on,tested_on,hyperparameter,target,prediction,bleu,rougeL,meteor,bertscore-f1,use
0,T5_nq_nq,T5,nq,nq,{'defaults': True},what was the real name of saudi arabia,what is the name of the new saudi kingdom,0.0000,0.5882,0.4593,0.9098,0.7510
1,T5_nq_nq,T5,nq,nq,{'defaults': True},whats the most liked picture on instagram 2018,what is the most liked picture on instagram,0.6804,0.7500,0.8648,0.9323,0.8849
2,T5_nq_nq,T5,nq,nq,{'defaults': True},where does the movie proof of life take place,where does the movie adol take place,0.3674,0.7500,0.6692,0.9107,0.7893
3,T5_nq_nq,T5,nq,nq,{'defaults': True},where is net profit on the balance sheet,where does net profit come from in a financial...,0.0000,0.3333,0.3117,0.8561,0.7280
4,T5_nq_nq,T5,nq,nq,{'defaults': True},when was fingerprinting first used by the police,when did fingerprint analysis become a common ...,0.0000,0.1250,0.1250,0.8250,0.6326
...,...,...,...,...,...,...,...,...,...,...,...,...
429430,bart_amalgam_quac_hyp,bart,amalgam,quac,"{'defaults': False, 'max_length': 50, 'beams':...",When was their last race against each other?,Did Poulidor win the Tour de France?,0.0000,0.0000,0.0562,0.6764,0.3188
429431,bart_amalgam_quac_hyp,bart,amalgam,quac,"{'defaults': False, 'max_length': 50, 'beams':...",Did any single on that album make the charts?,How did the Seekers do on the charts?,0.0000,0.4706,0.5163,0.8072,0.3574
429432,bart_amalgam_quac_hyp,bart,amalgam,quac,"{'defaults': False, 'max_length': 50, 'beams':...",When did Nick first get into music?,When did Nick Carter begin his acting career?,0.0000,0.4000,0.5506,0.8364,0.6590
429433,bart_amalgam_quac_hyp,bart,amalgam,quac,"{'defaults': False, 'max_length': 50, 'beams':...",What office was she trying to get?,When did Pirro announce she would run against ...,0.0000,0.1176,0.1205,0.7122,0.3013


### Find means for all models

In [ ]:
# Find the overall averages for each prediction set and metric
eval_means_df = evaluation_df.groupby(["nickname"]).mean()

# Reset index so able to groupby and sort below to find top scoring prediction sets for each metric
eval_means_df = eval_means_df.reset_index("nickname")

eval_means_df

,nickname,bleu,rouge1,rouge2,rougeL,rougeLsum,meteor,bertscore-precision,bertscore-recall,bertscore-f1,bleurt,use
0,T5_amalgam_nq,0.1613,0.5459,0.3218,0.5202,0.5202,0.4940,0.8601,0.8533,0.8564,-0.3562,0.7142
1,T5_amalgam_quac,0.0290,0.2573,0.1026,0.2522,0.2522,0.2551,0.7889,0.7823,0.7851,-0.9355,0.3984
2,T5_amalgam_squad,0.1252,0.4877,0.2688,0.4552,0.4552,0.4530,0.8691,0.8563,0.8621,-0.3265,0.6467
3,T5_amalgam_triviaqa,0.0587,0.4126,0.1946,0.3468,0.3468,0.3270,0.8371,0.8036,0.8191,-0.6840,0.5931
4,T5_nq_nq,0.1610,0.5440,0.3213,0.5191,0.5191,0.4916,0.8616,0.8523,0.8566,-0.3687,0.7126
5,T5_nq_quac,0.0010,0.1539,0.0339,0.1503,0.1503,0.1113,0.6927,0.7008,0.6964,-1.3609,0.2854
6,T5_nq_squad,0.0141,0.3576,0.1441,0.3301,0.3301,0.2593,0.8008,0.7729,0.7863,-0.7626,0.5397
7,T5_nq_triviaqa,0.0073,0.3140,0.1167,0.2731,0.2731,0.1860,0.7917,0.7388,0.7639,-1.0624,0.5125
8,T5_quac_nq,0.0002,0.1491,0.0239,0.1448,0.1448,0.0761,0.6952,0.6813,0.6880,-1.5072,0.2884
9,T5_quac_quac,0.0026,0.1720,0.0411,0.1710,0.1710,0.1653,0.7713,0.7500,0.7601,-1.3705,0.3211


### Create BART and T5 dataframes

In [ ]:
# Create BART dataframe
bart_df = evaluation_df[evaluation_df["base_model"] == "bart"]

# Create T5 dataframe
t5_df = evaluation_df[evaluation_df["base_model"] == "T5"]

## Metrics by Model

In [ ]:
df = clean_eval_df[default_hyperparameter_mask]
df.groupby(["base_model"]).mean()

,bleu,rougeL,meteor,bertscore-f1,use
base_model,,,,,
T5,0.0387,0.2784,0.2595,0.7829,0.4649
bart,0.0518,0.3142,0.2932,0.7977,0.5104


## Metrics by Data Tested On

In [ ]:
# Set number of decimal places for scores in pandas df
pd.set_option('precision', 4)

### NQ

In [ ]:
# Tested on NQ
nq_df = evaluation_df[(evaluation_df['tested_on'] == 'nq') & (evaluation_df['base_model'] == 'bart')].groupby(['base_model', 'trained_on', 'tested_on']).mean().sort_values(by=['bertscore-f1'], ascending=False)

nq_df

bleu  rouge1  rouge2  rougeL  rougeLsum  \
base_model trained_on tested_on                                              
bart       nq         nq         0.2035  0.5759  0.3570  0.5499     0.5499   
           amalgam    nq         0.1962  0.5698  0.3532  0.5451     0.5451   
           squad      nq         0.0118  0.4289  0.2126  0.4107     0.4107   
           triviaqa   nq         0.0113  0.4037  0.1982  0.3665     0.3665   
           quac       nq         0.0050  0.3354  0.1381  0.3273     0.3273   

                                 meteor  bertscore-precision  \
base_model trained_on tested_on                                
bart       nq         nq         0.5280               0.8705   
           amalgam    nq         0.5240               0.8664   
           squad      nq         0.3552               0.7929   
           triviaqa   nq         0.3683               0.7772   
           quac       nq         0.2257               0.7605   

                                 bertscore-recall  bertscore-f1  bleurt  \
base_model trained_on tested_on                                           
bart       nq         nq                   0.8644        0.8672 -0.2524   
           amalgam    nq                   0.8627        0.8643 -0.2673   
           squad      nq                   0.8051        0.7986 -0.6335   
           triviaqa   nq                   0.8163        0.7959 -0.7319   
           quac       nq                   0.7424        0.7511 -0.9865   

                                    use  
base_model trained_on tested_on          
bart       nq         nq         0.7364  
           amalgam    nq         0.7294  
           squad      nq         0.6304  
           triviaqa   nq         0.6243  
           quac       nq         0.5175

In [ ]:
# Convert to Latex
print(nq_df.to_latex(index=False)) 

\begin{tabular}{rrrrrrrrrrr}
\toprule
  bleu &  rouge1 &  rouge2 &  rougeL &  rougeLsum &  meteor &  bertscore-precision &  bertscore-recall &  bertscore-f1 &  bleurt &    use \\
0.2035 &  0.5759 &  0.3570 &  0.5499 &     0.5499 &  0.5280 &               0.8705 &            0.8644 &        0.8672 & -0.2524 & 0.7364 \\
\midrule
0.1962 &  0.5698 &  0.3532 &  0.5451 &     0.5451 &  0.5240 &               0.8664 &            0.8627 &        0.8643 & -0.2673 & 0.7294 \\
0.0118 &  0.4289 &  0.2126 &  0.4107 &     0.4107 &  0.3552 &               0.7929 &            0.8051 &        0.7986 & -0.6335 & 0.6304 \\
0.0113 &  0.4037 &  0.1982 &  0.3665 &     0.3665 &  0.3683 &               0.7772 &            0.8163 &        0.7959 & -0.7319 & 0.6243 \\
0.0050 &  0.3354 &  0.1381 &  0.3273 &     0.3273 &  0.2257 &               0.7605 &            0.7424 &        0.7511 & -0.9865 & 0.5175 \\
\bottomrule
\end{tabular}



### SQuAD

In [ ]:
# Tested on SQuAD
squad_df = evaluation_df[evaluation_df['tested_on'] == 'squad'].groupby(['base_model', 'trained_on', 'tested_on']).mean().sort_values(by=['bertscore-f1'], ascending=False)

squad_df

bleu  rouge1  rouge2  rougeL  rougeLsum  \
base_model trained_on tested_on                                              
bart       squad      squad      0.1515  0.5036  0.2905  0.4692     0.4692   
T5         squad      squad      0.1323  0.4842  0.2684  0.4521     0.4521   
bart       amalgam    squad      0.1362  0.4935  0.2798  0.4588     0.4588   
T5         amalgam    squad      0.1252  0.4877  0.2688  0.4552     0.4552   
bart       triviaqa   squad      0.0479  0.3519  0.1425  0.3114     0.3114   
           quac       squad      0.0279  0.3053  0.1131  0.2902     0.2902   
T5         triviaqa   squad      0.0285  0.2938  0.1059  0.2426     0.2426   
           nq         squad      0.0141  0.3576  0.1441  0.3301     0.3301   
bart       nq         squad      0.0144  0.3492  0.1393  0.3228     0.3228   
T5         quac       squad      0.0009  0.1800  0.0253  0.1734     0.1734   

                                 meteor  bertscore-precision  \
base_model trained_on tested_on                                
bart       squad      squad      0.4756               0.8752   
T5         squad      squad      0.4555               0.8714   
bart       amalgam    squad      0.4605               0.8690   
T5         amalgam    squad      0.4530               0.8691   
bart       triviaqa   squad      0.3387               0.8100   
           quac       squad      0.2453               0.8329   
T5         triviaqa   squad      0.2894               0.7839   
           nq         squad      0.2593               0.8008   
bart       nq         squad      0.2522               0.7976   
T5         quac       squad      0.1343               0.7693   

                                 bertscore-recall  bertscore-f1  bleurt  \
base_model trained_on tested_on                                           
bart       squad      squad                0.8651        0.8696 -0.2657   
T5         squad      squad                0.8598        0.8649 -0.3187   
bart       amalgam    squad                0.8580        0.8629 -0.3088   
T5         amalgam    squad                0.8563        0.8621 -0.3265   
bart       triviaqa   squad                0.8253        0.8169 -0.6442   
           quac       squad                0.7760        0.8028 -0.8311   
T5         triviaqa   squad                0.7999        0.7910 -0.8792   
           nq         squad                0.7729        0.7863 -0.7626   
bart       nq         squad                0.7713        0.7839 -0.7636   
T5         quac       squad                0.7168        0.7417 -1.3550   

                                    use  
base_model trained_on tested_on          
bart       squad      squad      0.6570  
T5         squad      squad      0.6422  
bart       amalgam    squad      0.6504  
T5         amalgam    squad      0.6467  
bart       triviaqa   squad      0.5354  
           quac       squad      0.4456  
T5         triviaqa   squad      0.4581  
           nq         squad      0.5397  
bart       nq         squad      0.5407  
T5         quac       squad      0.2571

In [ ]:
# Convert to Latex
print(squad_df.to_latex(index=False)) 

\begin{tabular}{rrrrrrrrrrr}
\toprule
  bleu &  rouge1 &  rouge2 &  rougeL &  rougeLsum &  meteor &  bertscore-precision &  bertscore-recall &  bertscore-f1 &  bleurt &    use \\
0.1515 &  0.5036 &  0.2905 &  0.4692 &     0.4692 &  0.4756 &               0.8752 &            0.8651 &        0.8696 & -0.2657 & 0.6570 \\
\midrule
0.1323 &  0.4842 &  0.2684 &  0.4521 &     0.4521 &  0.4555 &               0.8714 &            0.8598 &        0.8649 & -0.3187 & 0.6422 \\
0.1362 &  0.4935 &  0.2798 &  0.4588 &     0.4588 &  0.4605 &               0.8690 &            0.8580 &        0.8629 & -0.3088 & 0.6504 \\
0.1252 &  0.4877 &  0.2688 &  0.4552 &     0.4552 &  0.4530 &               0.8691 &            0.8563 &        0.8621 & -0.3265 & 0.6467 \\
0.0479 &  0.3519 &  0.1425 &  0.3114 &     0.3114 &  0.3387 &               0.8100 &            0.8253 &        0.8169 & -0.6442 & 0.5354 \\
0.0279 &  0.3053 &  0.1131 &  0.2902 &     0.2902 &  0.2453 &               0.8329 &            0.7760 &   

### TriviaQA

In [ ]:
# Tested on TriviaQA
triviaqa_df = evaluation_df[evaluation_df['tested_on'] == 'triviaqa'].groupby(['base_model', 'trained_on', 'tested_on']).mean().sort_values(by=['bertscore-f1'], ascending=False)

triviaqa_df

bleu  rouge1  rouge2  rougeL  rougeLsum  \
base_model trained_on tested_on                                              
bart       amalgam    triviaqa   0.0778  0.4355  0.2212  0.3684     0.3684   
           triviaqa   triviaqa   0.0766  0.4344  0.2193  0.3682     0.3682   
T5         amalgam    triviaqa   0.0587  0.4126  0.1946  0.3468     0.3468   
           triviaqa   triviaqa   0.0456  0.3780  0.1613  0.3153     0.3153   
           squad      triviaqa   0.0225  0.3098  0.1093  0.2667     0.2667   
bart       squad      triviaqa   0.0219  0.2975  0.1100  0.2602     0.2602   
           nq         triviaqa   0.0079  0.3317  0.1283  0.2862     0.2862   
T5         nq         triviaqa   0.0073  0.3140  0.1167  0.2731     0.2731   
bart       quac       triviaqa   0.0070  0.2180  0.0625  0.1971     0.1971   
T5         quac       triviaqa   0.0002  0.1248  0.0128  0.1162     0.1162   

                                 meteor  bertscore-precision  \
base_model trained_on tested_on                                
bart       amalgam    triviaqa   0.3541               0.8439   
           triviaqa   triviaqa   0.3513               0.8444   
T5         amalgam    triviaqa   0.3270               0.8371   
           triviaqa   triviaqa   0.2995               0.8257   
           squad      triviaqa   0.2255               0.8141   
bart       squad      triviaqa   0.2189               0.8151   
           nq         triviaqa   0.2008               0.7999   
T5         nq         triviaqa   0.1860               0.7917   
bart       quac       triviaqa   0.1505               0.7998   
T5         quac       triviaqa   0.0889               0.7418   

                                 bertscore-recall  bertscore-f1  bleurt  \
base_model trained_on tested_on                                           
bart       amalgam    triviaqa             0.8112        0.8264 -0.6272   
           triviaqa   triviaqa             0.8098        0.8259 -0.6317   
T5         amalgam    triviaqa             0.8036        0.8191 -0.6840   
           triviaqa   triviaqa             0.7966        0.8101 -0.7718   
           squad      triviaqa             0.7652        0.7881 -0.9154   
bart       squad      triviaqa             0.7641        0.7880 -0.9097   
           nq         triviaqa             0.7446        0.7708 -1.0172   
T5         nq         triviaqa             0.7388        0.7639 -1.0624   
bart       quac       triviaqa             0.7166        0.7553 -1.1698   
T5         quac       triviaqa             0.6726        0.7050 -1.5220   

                                    use  
base_model trained_on tested_on          
bart       amalgam    triviaqa   0.6088  
           triviaqa   triviaqa   0.6080  
T5         amalgam    triviaqa   0.5931  
           triviaqa   triviaqa   0.5797  
           squad      triviaqa   0.5026  
bart       squad      triviaqa   0.4917  
           nq         triviaqa   0.5345  
T5         nq         triviaqa   0.5125  
bart       quac       triviaqa   0.3959  
T5         quac       triviaqa   0.2256

In [ ]:
# Convert to Latex
print(triviaqa_df.to_latex(index=False)) 

\begin{tabular}{rrrrrrrrrrr}
\toprule
  bleu &  rouge1 &  rouge2 &  rougeL &  rougeLsum &  meteor &  bertscore-precision &  bertscore-recall &  bertscore-f1 &  bleurt &    use \\
0.0778 &  0.4355 &  0.2212 &  0.3684 &     0.3684 &  0.3541 &               0.8439 &            0.8112 &        0.8264 & -0.6272 & 0.6088 \\
\midrule
0.0766 &  0.4344 &  0.2193 &  0.3682 &     0.3682 &  0.3513 &               0.8444 &            0.8098 &        0.8259 & -0.6317 & 0.6080 \\
0.0587 &  0.4126 &  0.1946 &  0.3468 &     0.3468 &  0.3270 &               0.8371 &            0.8036 &        0.8191 & -0.6840 & 0.5931 \\
0.0456 &  0.3780 &  0.1613 &  0.3153 &     0.3153 &  0.2995 &               0.8257 &            0.7966 &        0.8101 & -0.7718 & 0.5797 \\
0.0225 &  0.3098 &  0.1093 &  0.2667 &     0.2667 &  0.2255 &               0.8141 &            0.7652 &        0.7881 & -0.9154 & 0.5026 \\
0.0219 &  0.2975 &  0.1100 &  0.2602 &     0.2602 &  0.2189 &               0.8151 &            0.7641 &   

### QuAC

In [ ]:
# Tested on QuAC
quac_df = evaluation_df[evaluation_df['tested_on'] == 'quac'].groupby(['base_model', 'trained_on', 'tested_on']).mean().sort_values(by=['bertscore-f1'], ascending=False)

quac_df

bleu  rouge1  rouge2  rougeL  rougeLsum  \
base_model trained_on tested_on                                              
bart       quac       quac       0.0582  0.2859  0.1407  0.2820     0.2820   
T5         amalgam    quac       0.0290  0.2573  0.1026  0.2522     0.2522   
bart       amalgam    quac       0.0281  0.2372  0.0881  0.2328     0.2328   
T5         quac       quac       0.0026  0.1720  0.0411  0.1710     0.1710   
           squad      quac       0.0062  0.1898  0.0451  0.1837     0.1837   
bart       squad      quac       0.0073  0.1887  0.0463  0.1836     0.1836   
           triviaqa   quac       0.0015  0.1216  0.0254  0.1134     0.1134   
T5         triviaqa   quac       0.0010  0.1099  0.0193  0.0989     0.0989   
bart       nq         quac       0.0011  0.1530  0.0347  0.1497     0.1497   
T5         nq         quac       0.0010  0.1539  0.0339  0.1503     0.1503   

                                 meteor  bertscore-precision  \
base_model trained_on tested_on                                
bart       quac       quac       0.2877               0.7958   
T5         amalgam    quac       0.2551               0.7889   
bart       amalgam    quac       0.2493               0.7582   
T5         quac       quac       0.1653               0.7713   
           squad      quac       0.2096               0.7355   
bart       squad      quac       0.2053               0.7364   
           triviaqa   quac       0.1631               0.6813   
T5         triviaqa   quac       0.1497               0.6817   
bart       nq         quac       0.1127               0.6924   
T5         nq         quac       0.1113               0.6927   

                                 bertscore-recall  bertscore-f1  bleurt  \
base_model trained_on tested_on                                           
bart       quac       quac                 0.7900        0.7924 -0.8071   
T5         amalgam    quac                 0.7823        0.7851 -0.9355   
bart       amalgam    quac                 0.7837        0.7701 -1.0411   
T5         quac       quac                 0.7500        0.7601 -1.3705   
           squad      quac                 0.7751        0.7543 -1.1584   
bart       squad      quac                 0.7731        0.7537 -1.2160   
           triviaqa   quac                 0.7408        0.7093 -1.3748   
T5         triviaqa   quac                 0.7362        0.7073 -1.3861   
bart       nq         quac                 0.7024        0.6971 -1.3418   
T5         nq         quac                 0.7008        0.6964 -1.3609   

                                    use  
base_model trained_on tested_on          
bart       quac       quac       0.4174  
T5         amalgam    quac       0.3984  
bart       amalgam    quac       0.3628  
T5         quac       quac       0.3211  
           squad      quac       0.3232  
bart       squad      quac       0.3181  
           triviaqa   quac       0.2428  
T5         triviaqa   quac       0.2271  
bart       nq         quac       0.2867  
T5         nq         quac       0.2854

In [ ]:
# Convert to Latex
print(quac_df.to_latex(index=False)) 

\begin{tabular}{rrrrrrrrrrr}
\toprule
  bleu &  rouge1 &  rouge2 &  rougeL &  rougeLsum &  meteor &  bertscore-precision &  bertscore-recall &  bertscore-f1 &  bleurt &    use \\
0.0582 &  0.2859 &  0.1407 &  0.2820 &     0.2820 &  0.2877 &               0.7958 &            0.7900 &        0.7924 & -0.8071 & 0.4174 \\
\midrule
0.0290 &  0.2573 &  0.1026 &  0.2522 &     0.2522 &  0.2551 &               0.7889 &            0.7823 &        0.7851 & -0.9355 & 0.3984 \\
0.0281 &  0.2372 &  0.0881 &  0.2328 &     0.2328 &  0.2493 &               0.7582 &            0.7837 &        0.7701 & -1.0411 & 0.3628 \\
0.0026 &  0.1720 &  0.0411 &  0.1710 &     0.1710 &  0.1653 &               0.7713 &            0.7500 &        0.7601 & -1.3705 & 0.3211 \\
0.0062 &  0.1898 &  0.0451 &  0.1837 &     0.1837 &  0.2096 &               0.7355 &            0.7751 &        0.7543 & -1.1584 & 0.3232 \\
0.0073 &  0.1887 &  0.0463 &  0.1836 &     0.1836 &  0.2053 &               0.7364 &            0.7731 &   

### Format code to PEP 8 Standards

### Steps

*   Install:

In [ ]:
!pip install black[jupyter]

*   To format your code run:

In [ ]:
!black /content/drive/MyDrive/'Colab Notebooks'/prediction_text_analysis.ipynb

All done! ✨ 🍰 ✨
1 file left unchanged.



*   Don't save your notebook, hit F5 (Command + r) to refresh the page
*   Voila!
*   Now save!